In [2]:
import pickle

with open('words_erp.pkl', 'rb') as file:
    loaded_content = pickle.load(file)

print(loaded_content)


["'s", 'Au', 'I', 'ID', 'USN', 'a', 'address', 'admission', 'app', 'appreciate', 'are', 'assessment', 'attendance', 'attendnce', 'auid', 'birth', 'book', 'built', 'bye', 'can', 'catch', 'change', 'check', 'class', 'classroom', 'collage', 'college', 'contact', 'could', 'course', 'coursera', 'created', 'daily', 'dance', 'date', 'day', 'department', 'detail', 'determine', 'did', 'displayed', 'do', 'email', 'emmergency', 'erp', 'exam', 'farewell', 'fee', 'feedback', 'find', 'for', 'from', 'give', 'going', 'good', 'goodbye', 'hand', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'history', 'how', 'i', 'ia', 'icecream', 'id', 'idiot', 'in', 'information', 'internal', 'internals', 'is', 'it', 'know', 'later', 'location', 'logout', 'love', 'made', 'mark', 'me', 'modify', 'much', 'my', 'name', 'need', 'nonsense', 'notification', 'now', 'number', 'of', 'parent', 'password', 'pay', 'payment', 'phone', 'please', 'proctor', 'profile', 'rate', 'receipt', 'record', 'reset', 'revoir', 'room', 'score

In [3]:
import pickle

with open('classes_erp.pkl', 'rb') as file:
    loaded_content = pickle.load(file)

print(loaded_content)

['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '3', '4', '5', '6', '7', '8', '9', 'attendance_check', 'attendance_update', 'classroom_location', 'creation', 'general_query', 'goodbye', 'greeting', 'help', 'proctor_details', 'student_details_update', 'thanks']


In [6]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intent(1).json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',', ':', '-']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words_erp.pkl', 'wb'))
pickle.dump(classes, open('classes_erp.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

X = training[:, :len(words)]
Y = training[:, len(words):]

# Number of folds for cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(X, np.argmax(Y, axis=1))):
    print(f"Training Fold {fold + 1}...")

    trainX, valX = X[train_index], X[val_index]
    trainY, valY = Y[train_index], Y[val_index]

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, input_shape=(len(trainX[0]),), activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(trainX, trainY, epochs=500, batch_size=32, verbose=1, validation_data=(valX, valY))

# Save the model after cross-validation
model.save('chatbot_erp.h5')
print('Done')


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Training Fold 1...
Epoch 1/500
5/5 [==============================] - 2s 125ms/step - loss: 3.3911 - accuracy: 0.0584 - val_loss: 3.3506 - val_accuracy: 0.2286
Epoch 2/500
5/5 [==============================] - 0s 41ms/step - loss: 3.3063 - accuracy: 0.1095 - val_loss: 3.2824 - val_accuracy: 0.1714
Epoch 3/500
5/5 [==============================] - 0s 45ms/step - loss: 3.1800 - accuracy: 0.1825 - val_loss: 3.2068 - val_accuracy: 0.2000
Epoch 4/500
5/5 [==============================] - 0s 44ms/step - loss: 3.0711 - accuracy: 0.1752 - val_loss: 3.1322 - val_accuracy: 0.2000
Epoch 5/500
5/5 [==============================] - 0s 43ms/step - loss: 3.0192 - accuracy: 0.1825 - val_loss: 3.0615 - val_accuracy: 0.2000
Epoch 6/500
5/5 [==============================] - 0s 45ms/step - loss: 2.8317 - accuracy: 0.1898 - val_loss: 2.9890 - val_accuracy: 0.1714
Epoch 7/500
5/5 [==============================] - 0s 50ms/step - loss: 2.7442 - accuracy: 0.2263 - val_loss: 2.9206 - val_accuracy: 0.2286


5/5 [==============================] - 0s 45ms/step - loss: 0.0971 - accuracy: 0.9635 - val_loss: 2.3031 - val_accuracy: 0.4571
Epoch 60/500
5/5 [==============================] - 0s 49ms/step - loss: 0.1512 - accuracy: 0.9343 - val_loss: 2.2783 - val_accuracy: 0.4571
Epoch 61/500
5/5 [==============================] - 0s 42ms/step - loss: 0.1497 - accuracy: 0.9489 - val_loss: 2.2745 - val_accuracy: 0.4571
Epoch 62/500
5/5 [==============================] - 0s 49ms/step - loss: 0.1653 - accuracy: 0.9270 - val_loss: 2.2889 - val_accuracy: 0.4286
Epoch 63/500
5/5 [==============================] - 0s 55ms/step - loss: 0.1474 - accuracy: 0.9343 - val_loss: 2.3409 - val_accuracy: 0.4286
Epoch 64/500
5/5 [==============================] - 0s 51ms/step - loss: 0.1531 - accuracy: 0.9416 - val_loss: 2.4421 - val_accuracy: 0.4286
Epoch 65/500
5/5 [==============================] - 0s 42ms/step - loss: 0.1374 - accuracy: 0.9635 - val_loss: 2.4894 - val_accuracy: 0.4571
Epoch 66/500
5/5 [========

5/5 [==============================] - 0s 48ms/step - loss: 0.0716 - accuracy: 0.9635 - val_loss: 2.6157 - val_accuracy: 0.4857
Epoch 118/500
5/5 [==============================] - 0s 42ms/step - loss: 0.0951 - accuracy: 0.9489 - val_loss: 2.6350 - val_accuracy: 0.4857
Epoch 119/500
5/5 [==============================] - 0s 42ms/step - loss: 0.1453 - accuracy: 0.9343 - val_loss: 2.6629 - val_accuracy: 0.4857
Epoch 120/500
5/5 [==============================] - 0s 47ms/step - loss: 0.0839 - accuracy: 0.9489 - val_loss: 2.7027 - val_accuracy: 0.4857
Epoch 121/500
5/5 [==============================] - 0s 52ms/step - loss: 0.1208 - accuracy: 0.9416 - val_loss: 2.7205 - val_accuracy: 0.4857
Epoch 122/500
5/5 [==============================] - 0s 49ms/step - loss: 0.0796 - accuracy: 0.9781 - val_loss: 2.7149 - val_accuracy: 0.4857
Epoch 123/500
5/5 [==============================] - 0s 45ms/step - loss: 0.1433 - accuracy: 0.9270 - val_loss: 2.7003 - val_accuracy: 0.4857
Epoch 124/500
5/5 [=

Epoch 175/500
5/5 [==============================] - 0s 48ms/step - loss: 0.0769 - accuracy: 0.9635 - val_loss: 2.9548 - val_accuracy: 0.4571
Epoch 176/500
5/5 [==============================] - 0s 42ms/step - loss: 0.1050 - accuracy: 0.9489 - val_loss: 3.0163 - val_accuracy: 0.4286
Epoch 177/500
5/5 [==============================] - 0s 46ms/step - loss: 0.1146 - accuracy: 0.9416 - val_loss: 3.0611 - val_accuracy: 0.4286
Epoch 178/500
5/5 [==============================] - 0s 52ms/step - loss: 0.0932 - accuracy: 0.9562 - val_loss: 3.0455 - val_accuracy: 0.4286
Epoch 179/500
5/5 [==============================] - 0s 51ms/step - loss: 0.1340 - accuracy: 0.9416 - val_loss: 3.0026 - val_accuracy: 0.4286
Epoch 180/500
5/5 [==============================] - 0s 46ms/step - loss: 0.1149 - accuracy: 0.9562 - val_loss: 2.9762 - val_accuracy: 0.4286
Epoch 181/500
5/5 [==============================] - 0s 46ms/step - loss: 0.0869 - accuracy: 0.9562 - val_loss: 2.9636 - val_accuracy: 0.4286
Epoch 

...

Epoch 379/500
5/5 [==============================] - 0s 43ms/step - loss: 0.0595 - accuracy: 0.9565 - val_loss: 2.4964 - val_accuracy: 0.6176
Epoch 380/500
5/5 [==============================] - 0s 44ms/step - loss: 0.0708 - accuracy: 0.9565 - val_loss: 2.4838 - val_accuracy: 0.6176
Epoch 381/500
5/5 [==============================] - 0s 48ms/step - loss: 0.0685 - accuracy: 0.9638 - val_loss: 2.4820 - val_accuracy: 0.6176
Epoch 382/500
5/5 [==============================] - 0s 47ms/step - loss: 0.0564 - accuracy: 0.9710 - val_loss: 2.4816 - val_accuracy: 0.6176
Epoch 383/500
5/5 [==============================] - 0s 54ms/step - loss: 0.0734 - accuracy: 0.9565 - val_loss: 2.4688 - val_accuracy: 0.6176
Epoch 384/500
5/5 [==============================] - 0s 43ms/step - loss: 0.0692 - accuracy: 0.9348 - val_loss: 2.4485 - val_accuracy: 0.6176
Epoch 385/500
5/5 [==============================] - 0s 42ms/step - loss: 0.0614 - accuracy: 0.9638 - val_loss: 2.4305 - val_accuracy: 0.6176
Epoch 

Epoch 437/500
5/5 [==============================] - 0s 52ms/step - loss: 0.0670 - accuracy: 0.9565 - val_loss: 2.5881 - val_accuracy: 0.6176
Epoch 438/500
5/5 [==============================] - 0s 50ms/step - loss: 0.0608 - accuracy: 0.9565 - val_loss: 2.5659 - val_accuracy: 0.6471
Epoch 439/500
5/5 [==============================] - 0s 46ms/step - loss: 0.0543 - accuracy: 0.9710 - val_loss: 2.5457 - val_accuracy: 0.6471
Epoch 440/500
5/5 [==============================] - 0s 39ms/step - loss: 0.0402 - accuracy: 1.0000 - val_loss: 2.5440 - val_accuracy: 0.6471
Epoch 441/500
5/5 [==============================] - 0s 48ms/step - loss: 0.0513 - accuracy: 0.9928 - val_loss: 2.5593 - val_accuracy: 0.6471
Epoch 442/500
5/5 [==============================] - 0s 47ms/step - loss: 0.0452 - accuracy: 0.9855 - val_loss: 2.5763 - val_accuracy: 0.6176
Epoch 443/500
5/5 [==============================] - 0s 43ms/step - loss: 0.0544 - accuracy: 0.9638 - val_loss: 2.5868 - val_accuracy: 0.6176
Epoch 

Epoch 495/500
5/5 [==============================] - 0s 43ms/step - loss: 0.0512 - accuracy: 0.9565 - val_loss: 2.6828 - val_accuracy: 0.6471
Epoch 496/500
5/5 [==============================] - 0s 44ms/step - loss: 0.0680 - accuracy: 0.9565 - val_loss: 2.6948 - val_accuracy: 0.6471
Epoch 497/500
5/5 [==============================] - 0s 39ms/step - loss: 0.0471 - accuracy: 0.9783 - val_loss: 2.7250 - val_accuracy: 0.6471
Epoch 498/500
5/5 [==============================] - 0s 42ms/step - loss: 0.0640 - accuracy: 0.9493 - val_loss: 2.7467 - val_accuracy: 0.6471
Epoch 499/500
5/5 [==============================] - 0s 49ms/step - loss: 0.0387 - accuracy: 0.9783 - val_loss: 2.7567 - val_accuracy: 0.6471
Epoch 500/500
5/5 [==============================] - 0s 52ms/step - loss: 0.0519 - accuracy: 0.9710 - val_loss: 2.7600 - val_accuracy: 0.6471


C:\Users\pragati\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done
